In [141]:
country_name_rus = {"Afganistan":'Афганистан',"China":'КНР',"CAR":'Центрально-Африканская Республика',
                    "Ethiopia":"Эфиопия ","India":'Индия', "Iran":'Иран',"KNDR":'КНДР', "Libya":'Ливия',
                    "Morocco":'Морокко', "Sudan":'Судан', "Syria":'Сирия', "Turkey":'Турция',
                    "Argentina":'Аргентина', "DRK_Congo":'ДРК (Конго)', "Burkina Faso":'Буркина-Фасо',
                    "Niger":'Нигер', "Mali":'Мали', "Senegal":'Сенегал', "Egipet":'Египет', "Gvineya":'Гвинея',
                    "Venesuela":'Венесуэла', "Algeria":'Алжир', "Zimbabve":'Зимбабве', "Tunis":'Тунис',
                    "Boliviya":'Боливия', "Braziliya":'Бразилия', "Yuar":'ЮАР', "Burundi":'Бурунди',
                    "Ruanda":'Руанда', "Mongoliya":'Монголия', "Respublika-koreya": 'Южная Корея',
                    "Chili": 'Чили',  'Bangladesh':'Бангладеш', 'Benin':'Бенин', 'Gabon':'Габон',
                    'Ekvatorialnaya-gviney':'Экваториальная Гвинея', 'Keniya':'Кения',
                    'Indoneziya':'Индонезия','Iordaniya':'Иордания', 'Irak':'Ирак',
                    'Kambodzha':'Камбоджа', 'Kuba':'Куба', 'Tailand':'Таиланд', 'Urugvay':'Уругвай',
                    'Kamerun':'Камерун', 'Madagaskar':'Мадагаскар', 'Mozambik':'Мозамбик', 'Chad':'Чад',
                    'Yaponiya':'Япония', 'Mexico': 'Мексика', 'Laos': 'Лаос', 'Pakistan': 'Пакистан', 
                    'Oae': 'ОАЭ', 'Saudovskaya_araviya': 'Саудовская Аравия','Yemen':'Йемен', 'RK_Congo':'Республика Конго'
                   }

CL =[ "Afganistan","China","CAR", "Ethiopia",
      "India", "Iran","KNDR", "Libya", "Morocco", "Sudan",
      "Syria", "Turkey", "Argentina", "DRK_Congo", "Burkina Faso",
      "Niger", "Mali", "Senegal", "Egipet", "Gvineya", "Venesuela",
      "Algeria", "Zimbabve", "Tunis", "Boliviya", "Braziliya",
      "Yuar", "Burundi", "Ruanda", "Mongoliya", 'Pakistan', 
     "Chili", 'Bangladesh', 'Benin', 'Gabon', 'Laos', 'Chad' , 
     'Keniya', 'Indoneziya','Iordaniya', 'Irak', 'Kambodzha', 'Kuba',
     'Tailand','Urugvay', 'Kamerun', 'Madagaskar', 'Mozambik', 'Yaponiya',
     'Mexico', 'Oae','Saudovskaya_araviya', 'Yemen', 'Pakistan', 'Laos', 'RK_Congo'
    ]

In [142]:
country_name_rus_T = {}
for item in country_name_rus.keys():
    country_name_rus_T[country_name_rus[item]] = item

print (country_name_rus_T)

{'Афганистан': 'Afganistan', 'КНР': 'China', 'Центрально-Африканская Республика': 'CAR', 'Эфиопия ': 'Ethiopia', 'Индия': 'India', 'Иран': 'Iran', 'КНДР': 'KNDR', 'Ливия': 'Libya', 'Морокко': 'Morocco', 'Судан': 'Sudan', 'Сирия': 'Syria', 'Турция': 'Turkey', 'Аргентина': 'Argentina', 'ДРК (Конго)': 'DRK_Congo', 'Буркина-Фасо': 'Burkina Faso', 'Нигер': 'Niger', 'Мали': 'Mali', 'Сенегал': 'Senegal', 'Египет': 'Egipet', 'Гвинея': 'Gvineya', 'Венесуэла': 'Venesuela', 'Алжир': 'Algeria', 'Зимбабве': 'Zimbabve', 'Тунис': 'Tunis', 'Боливия': 'Boliviya', 'Бразилия': 'Braziliya', 'ЮАР': 'Yuar', 'Бурунди': 'Burundi', 'Руанда': 'Ruanda', 'Монголия': 'Mongoliya', 'Южная Корея': 'Respublika-koreya', 'Чили': 'Chili', 'Бангладеш': 'Bangladesh', 'Бенин': 'Benin', 'Габон': 'Gabon', 'Экваториальная Гвинея': 'Ekvatorialnaya-gviney', 'Кения': 'Keniya', 'Индонезия': 'Indoneziya', 'Иордания': 'Iordaniya', 'Ирак': 'Irak', 'Камбоджа': 'Kambodzha', 'Куба': 'Kuba', 'Таиланд': 'Tailand', 'Уругвай': 'Urugvay', 'К

In [143]:
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 


In [144]:
stemming=True # Stemming control
if stemming:
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("russian")

In [145]:
def corpus_ngram_replace (corpus, ng_count_target=7):
    # ng_count_target=7 минимальная частота в корпусе
    from sklearn.feature_extraction.text import CountVectorizer
    
    count_ngram = CountVectorizer(ngram_range=(2,3))
    X_train_transformed_ngram = count_ngram.fit_transform(corpus)
    vocab = count_ngram.vocabulary_
    count_values = X_train_transformed_ngram.toarray().sum(axis=0)
    ng_text_feq =[]
    for ng_count, ng_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
        if ng_count>ng_count_target:
            print(ng_count, ng_text)
            ng_text_feq.append(ng_text)
    print(len(ng_text_feq))
    corpus_with_ngram = []
    for text in x_temp:    
        for item in ng_text_feq:
            text = text.replace(item, item.replace(' ', '_'))        
        corpus_with_ngram.append(text)
    return corpus_with_ngram

In [146]:
ru_stopwords = set(
        """
    а авось ага агу аж ай али алло ау ах ая
    б будем будет будете будешь буду будут будучи будь будьте бы был была были было
    быть бац без безусловно бишь благо благодаря ближайшие близко более больше
    будто бывает бывала бывали бываю бывают бытует
    в вам вами вас весь во вот все всё всего всей всем всём всеми всему всех всею
    всея всю вся вы ваш ваша ваше ваши вдали вдобавок вдруг ведь везде вернее
    взаимно взаправду видно вишь включая вместо внакладе вначале вне вниз внизу
    вновь вовсе возможно воистину вокруг вон вообще вопреки вперекор вплоть
    вполне вправду вправе впрочем впрямь вресноту вроде вряд всегда всюду
    всякий всякого всякой всячески вчеред
    г го где гораздо гав
    д да для до дабы давайте давно давным даже далее далеко дальше данная
    данного данное данной данном данному данные данный данных дану данунах
    даром де действительно довольно доколе доколь долго должен должна
    должно должны должный дополнительно другая другие другим другими
    других другое другой
    е его едим едят ее её ей ел ела ем ему емъ если ест есть ешь еще ещё ею едва
    ежели еле
    ж же
    з за затем зато зачем здесь значит зря
    и из или им ими имъ их ибо иль имеет имел имела имело именно иметь иначе
    иногда иным иными итак ишь
    й
    к как кем ко когда кого ком кому комья которая которого которое которой котором
    которому которою которую которые который которым которыми которых кто ка кабы
    каждая каждое каждые каждый кажется казалась казались казалось казался казаться
    какая какие каким какими каков какого какой какому какою касательно кой коли
    коль конечно короче кроме кстати ку куда
    л ли либо лишь любая любого любое любой любом любую любыми любых
    м меня мне мной мною мог моги могите могла могли могло могу могут мое моё моего
    моей моем моём моему моею можем может можете можешь мои мой моим моими моих
    мочь мою моя мы мало меж между менее меньше мимо многие много многого многое
    многом многому можно мол му
    н на нам нами нас наса наш наша наше нашего нашей нашем нашему нашею наши нашим
    нашими наших нашу не него нее неё ней нем нём нему нет нею ним ними них но
    наверняка наверху навряд навыворот над надо назад наиболее наизворот
    наизнанку наипаче накануне наконец наоборот наперед наперекор наподобие
    например напротив напрямую насилу настоящая настоящее настоящие настоящий
    насчет нате находиться начала начале неважно негде недавно недалеко незачем
    некем некогда некому некоторая некоторые некоторый некоторых некто некуда
    нельзя немногие немногим немного необходимо необходимости необходимые
    необходимым неоткуда непрерывно нередко несколько нету неужели нечего
    нечем нечему нечто нешто нибудь нигде ниже низко никак никакой никем
    никогда никого никому никто никуда ниоткуда нипочем ничего ничем ничему
    ничто ну нужная нужно нужного нужные нужный нужных ныне нынешнее нынешней
    нынешних нынче
    о об один одна одни одним одними одних одно одного одной одном одному одною
    одну он она оне они оно от оба общую обычно ого однажды однако ой около оный
    оп опять особенно особо особую особые откуда отнелижа отнелиже отовсюду
    отсюда оттого оттот оттуда отчего отчему ох очевидно очень ом
    п по при паче перед под подавно поди подобная подобно подобного подобные
    подобный подобным подобных поелику пожалуй пожалуйста позже поистине
    пока покамест поколе поколь покуда покудова помимо понеже поприще пор
    пора посему поскольку после посреди посредством потом потому потомушта
    похожем почему почти поэтому прежде притом причем про просто прочего
    прочее прочему прочими проще прям пусть
    р ради разве ранее рано раньше рядом
    с сам сама сами самим самими самих само самого самом самому саму свое своё
    своего своей своем своём своему своею свои свой своим своими своих свою своя
    себе себя собой собою самая самое самой самый самых сверх свыше се сего сей
    сейчас сие сих сквозь сколько скорее скоро следует слишком смогут сможет
    сначала снова со собственно совсем сперва спокону спустя сразу среди сродни
    стал стала стали стало стать суть сызнова
    та то ту ты ти так такая такие таким такими таких такого такое такой таком такому такою
    такую те тебе тебя тем теми тех тобой тобою того той только том томах тому
    тот тою также таки таков такова там твои твоим твоих твой твоя твоё
    теперь тогда тоже тотчас точно туда тут тьфу тая
    у уже увы уж ура ух ую
    ф фу
    х ха хе хорошо хотел хотела хотелось хотеть хоть хотя хочешь хочу хуже
    ч чего чем чём чему что чтобы часто чаще чей через чтоб чуть чхать чьим
    чьих чьё чё
    ш ша
    щ ща щас
    ы ых ые ый
    э эта эти этим этими этих это этого этой этом этому этот этою эту эдак эдакий
    эй эка экий этак этакий эх
    ю
    я явно явных яко якобы якоже
    
    и что не это  быть этот это свой как - этот весь быть что ▫ но г. % 
    
    afgbezparandzhi argentinarusa irandezhurniy info raspp аль privetfidel sex_drugs_kahlo full_text
    maslovasia china80s raspp_info prchand awaken_dragon asiatica_ru zangaro 
    africablack natasakado_official dnobangui africafordummies westernafrica meskob catcherinsudan africanists 
    syriaagency turk_gambit_ca shatergaddafi turkkulubu mideastr tangermanar marocrus arabstatesofgulf assadstash meastru strana_tuaregov 
    pqntc1 privetfidel altimurla favelasemrus pincheponchito laprimaveradelpatriarca lat_america tupireport venezuelanewsnetwork sputnikmundo 
    rt_russian infantmilitario IranPakistanAfganistan afgbezparandzhi DPRK_KNDR welcomeDPRK  irandezhurniy paxIranica 
    afgbezparandzhi Ethiopia_tezeta meskob india_tv2020 ninerasas indiareads speciallassi indiaanalytics Indosphere India_sangrahalaya 
    turkeyabout ShaterGaddafi tangermanar marocrus NovostiDamask dnobangui CatcherInSudan argentinarusa  Bras24 gdrusbf randentans novostimexico
    bengazi_exit

    
    """.split()
    )

In [147]:
def predict_contry (Text_post):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import cross_val_score
    import pickle

    ngram_replace = True # использование замены в тексте ngram
    
    with open('contry_post-model_ngram_rf_RandomForest_V3_015_03-10-2024.bin', 'rb') as f_out:
        rf_RandomForest, vocabl=pickle.load( f_out)
    x_temp = []
    content = str(Text_post).lower()  
    content = remove_emojis(content)
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    for i in range(1, 10):
        content = content.replace('  ', ' ')
    
    x_temp.append(content)



    count = CountVectorizer(ngram_range=(1,2), vocabulary = vocabl)
    X_t = count.transform(x_temp)
    accuracy = rf_RandomForest.predict_proba(X_t)
    ix = accuracy.argmax(1).item()
    # print(f'predicted class = {log_clf.classes_[ix]} and confidence = {accuracy[0,ix]:.2%}')
    predicted_class =rf_RandomForest.classes_[ix]
    confidence = accuracy[0,ix]    
    return predicted_class, confidence

In [148]:


def get_telegram_adv(user_name, Dir_Path= 'Set_A', iter_number=50, text_limit=True):
    '''
    :param user_name - Название телеграм канала:
    :param iter_number - Количество иттераций по дате публикации постов каждый шаг 20 постов max:
    :return:
    :text (str) - Текст постов канала (очищенный)
    web_names_set - список ссылок на внешние ресурсы, на которые ссылается канал
    telegram_names_set - список телеграм каналов на которые ссылантся канал
    set(lang) (set) - язык постов
    av_post_len - среднее значение символов в одном посте
    '''


    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    import re
    import os
    import io
    import json



    url = 'https://t.me/s/{}'.format(user_name)
    useragent = {'User-Agent': UserAgent().random}
    text = ''

    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 10))
        text = get_text(r, user_name, Dir_Path)
        number = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
        # print(number)
        if not len(number) == 0:
            last_post_num = int(number[-1])
            post_num = last_post_num
            if last_post_num < (iter_number+1)*20:
                iter_number = last_post_num//20 - 1
            for items in range(iter_number):
                r = requests.get('https://t.me/s/{}?before={}'.format(user_name, post_num), headers=useragent)
                get_text(r,user_name + str(post_num), Dir_Path)
                time.sleep(random.randrange(1, 7))
                temp_posts = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
                post_num = int(temp_posts[0])-1

    except Exception as ex:
        print(ex)
        return

def get_text(r, filename, Dir_Path):
    # Извлечение текста
    from bs4 import BeautifulSoup
    import re
    import os
    
    text = ''
    text_post = ''
    soup = BeautifulSoup(r.text, 'lxml')
    popular = soup.find_all('div', {'class': 'tgme_widget_message_text'})
    iterr = 0
    for item in popular:
        #item.string = re.sub('<br/><br/>', ' ', str(item))
        item.string = re.sub('<br/>', ' ', str(item))
        #tag.string = '<br/><br/>'
        #item.string.insert_before(tag)
        soup_temp = BeautifulSoup(item.text, 'lxml')
        #print (item)
        #print (soup_temp.text)
        text_post = re.sub(r'https?:\/\/.*[\r\n]*', ' ', soup_temp.text, flags=re.MULTILINE)
        #print (text_post.replace(r'#[^\s|\n]+',' '))
        text_post = re.sub(r'#[^\s|\n]+',' ', text_post)
        text_post = remove_emojis(text_post)
        text_post = text_post.replace('Тут была французская Западная Африка | Подписаться',' ')
        text_post = text_post.replace('Африканская инициатива | Подписаться', ' ')
        text_post = text_post.replace('@sex_drugs_kahlo — латиноамериканское счастье [нет]', ' ')
        text_post = text_post.replace('/Синьхуа/', ' ')
        text_post = text_post.replace('[Full Text]', ' ')
        text_post = text_post.replace(filename, ' ')
        text = text + '\n '+ text_post 
        predicted_class = "None"
        confidence = 0.
        try:
            if text_post != '':
                predicted_class, confidence = predict_contry (text_post)
                #print(predict_contry (text_post))
                print('predicted_class = {}, confidence_class = {:.0f}%, Длина: {}'.format(predicted_class, confidence*100,len(text_post)))
                print(lang_detect(text_post))
                #if predicted_class != item or confidence_class <0.2:
                #    print(content)
        except Exception as ex:
            print(ex)   
        
        #print('RES -{}, {}, {}, {}%'.format((predicted_class==Dir_Path) and (confidence>0.6), predicted_class, Dir_Path, confidence))
        if (len(text_post) > 150) and (len(text_post) < 20000) and (lang_detect(text_post) == 'ru') and (confidence>0.55):
            try:
                if not os.path.exists('./datasets/telegram_Dataset/{}'.format(country_name_rus_T[predicted_class])):
                    os.makedirs('./datasets/telegram_Dataset/{}'.format(country_name_rus_T[predicted_class]))
                with open('./datasets/telegram_Dataset/{}/@{}_telegram.txt'.format(country_name_rus_T[predicted_class], filename+'_'+str(iterr)), "w", encoding="utf-8") as some_file:
                    print(text_post, file=some_file)
                    some_file.close()
                    iterr = iterr+1
            except Exception as ex:
                print(ex)
    return text

def lang_detect (text = ''):
    # Определение языка канала
    from bs4 import BeautifulSoup
    from langdetect import detect
    import re

    if text !='':
        try:
            lang = detect(text[400:])
            #print('Язык поста {}'.format(lang))
            return lang
        except Exception as ex:
            #print(ex)
            return 'no text'
    else:
        return 'no text'

def pipline(content):
    import string
    import re
    from tokenizer_exceptions import normalizer_exc_rus
    print(content[:200])
    content = normalizer_exc_rus(content).lower()
    spec_chars = string.punctuation + '\xa0«»\t—…'
    content = re.sub('\n', ' ', content)
    content = re.sub('\r', ' ', content)
    
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    content = " ".join(content.split())
    print(content[:200])
    return content

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content




def main():
    import os


    next_iter_names = {
        # 'china': ['maslovasia', 'china80s', 'raspp_info', 'prchand', 'awaken_dragon', 'asiatica_ru'],
        # 'africa': ['zangaro', 'africablack', 'natasakado_official', 'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists'],
        # 'middleeast': ['syriaagency', 'turk_gambit_ca', 'shatergaddafi', 'turkkulubu', 'mideastr', 'tangermanar', 'marocrus', 'arabstatesofgulf', 'assadstash', 'meastru', 'strana_tuaregov']
        # 'latamerica': ['pqntc1', 'privetfidel', 'altimurla','favelasemrus', 'pincheponchito', 'laprimaveradelpatriarca', 'lat_america', 'tupireport', 'venezuelanewsnetwork', 'sputnikmundo']
        # 'news_ru': ['boris_rozhin', 'rt_russian', 'infantmilitario', 'svarschiki', 'divgen'],
        # 'news_ua': ['rezident_ua', 'taynaya_kantselyariya', 'the_military_analytics', 'spletnicca'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan', 'afgbezparandzhi'],
        #'KNDR': ['RusEmbDPRK'],
        #'Iran': ['irandezhurniy', 'paxIranica', 'IranPakistanAfganistan'],
        #'Afganistan': ['afgbezparandzhi', 'IranPakistanAfganistan'],
        #'Ethiopia': ['Ethiopia_tezeta', 'meskob'],
        #'Pakistan': ['IranPakistanAfganistan'],
        #'China': ['china80s', 'raspp_info', 'prchand'],
        #'India': ['india_tv2020', 'ninerasas', 'indiareads', 'speciallassi', 'indiaanalytics', 'Indosphere', 'India_sangrahalaya'],
        #'Turkey': ['turkeyabout', 'turkkulubu'],
        #'Libya': ['ShaterGaddafi'],
        #'Morocco': ['tangermanar', 'marocrus'],
        #'Syria': ['NovostiDamask'],
        #'CAR': ['dnobangui'],# Центрально-африканская Республика
        #'Sudan': ['CatcherInSudan'],
        #'Argentina': ['argentinarusa'],
        #'Iran': ['IranPakistanAfganistan'],
        #'Afganistan': [ 'IranPakistanAfganistan'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan'],
        #'africa': ['africaninitiative', 'africablack', 'natasakado_official', 'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists'],
        #'africa': ['africaninitiative'],
        #'China': ['xinhua_news_agency_ru'],
        #'Japan': ['Ruslan_in_Japan'],
        
        
        
        #'latamerica': ['pqntc1', 'privetfidel', 'altimurla','favelasemrus', 'pincheponchito','laprimaveradelpatriarca', 'lat_america', 'argentinarusa'],
        #'africa': ['zangaro', 'africablack',  'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists','africaninitiative'],
        #'China': ['xinhua_news_agency_ru',  'china80s', 'raspp_info', 'prchand', 'awaken_dragon', 'asiatica_ru'],
        #'middleeast': ['syriaagency', 'turk_gambit_ca', 'shatergaddafi', 'turkkulubu', 'mideastr', 'tangermanar', 'marocrus', 'arabstatesofgulf', 'assadstash', 'meastru', 'strana_tuaregov'],
        #'news_ru': ['rt_russian', 'infantmilitario'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan', 'afgbezparandzhi'],
        #'KNDR': ['RusEmbDPRK', 'dprk_kndr', 'welcomedprk'],
        #'Iran': ['irandezhurniy', 'paxIranica'],        
        #'Ethiopia': ['Ethiopia_tezeta', 'meskob'],
        #'India': ['india_tv2020', 'ninerasas', 'speciallassi', 'indiaanalytics', 'Indosphere', 'India_sangrahalaya'],
        #'Turkey': ['turkeyabout', 'turkkulubu'],
        #'Libya': ['ShaterGaddafi'],
        #'Morocco': ['tangermanar', 'marocrus'],
        #'Syria': ['NovostiDamask'],
        
        
        #'DRK_Congo': ['randentans'],
        #'Brasil': ['Bras24'],        
        #'mexico':['novostimexico'],
        #'Algeria': ['alg_today'],
        #'Tailand':['x_island'],
        'arab':['arabicafrica'],
        'Libya': ['bengazi_exit','ShaterGaddafi'],
        'Morocco': ['tangermanar', 'marocrus'],
        
        }





    #for country in CL:
    #    if not os.path.exists('./datasets/telegram_Dataset/{}'.format(country)):
    #        os.makedirs('./datasets/telegram_Dataset/{}'.format(country))
       
    for item in next_iter_names.keys():
        for items in next_iter_names[item]:
            print('Канал {}'.format(items))
            get_telegram_adv(items, item)


In [149]:
main()

Канал arabicafrica
predicted_class = Индонезия, confidence_class = 24%, Длина: 96
no text
predicted_class = Индонезия, confidence_class = 24%, Длина: 454
bg
predicted_class = Индонезия, confidence_class = 22%, Длина: 397
no text
predicted_class = Индонезия, confidence_class = 21%, Длина: 138
no text
predicted_class = Индонезия, confidence_class = 21%, Длина: 138
no text
predicted_class = Индонезия, confidence_class = 24%, Длина: 149
no text
predicted_class = Индонезия, confidence_class = 24%, Длина: 113
no text
predicted_class = Индонезия, confidence_class = 23%, Длина: 262
no text
predicted_class = Индонезия, confidence_class = 22%, Длина: 225
no text
predicted_class = Индонезия, confidence_class = 19%, Длина: 228
no text
predicted_class = Камерун, confidence_class = 48%, Длина: 313
no text
predicted_class = Индонезия, confidence_class = 24%, Длина: 250
no text
predicted_class = Индонезия, confidence_class = 23%, Длина: 278
no text
predicted_class = Индонезия, confidence_class = 16%, 

In [150]:
import sys
import os
import hashlib

def chunk_reader(fobj, chunk_size=1024):
    """Generator that reads a file in chunks of bytes"""
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk

def check_for_duplicates(paths, hash=hashlib.sha1):
    hashes = {}
    #print (os.listdir(paths))        
    for filename in [name for name in sorted(os.listdir(paths)) ]:
        full_path = os.path.join(paths, filename)
        hashobj = hash()
        for chunk in chunk_reader(open(full_path, 'rb')):
            hashobj.update(chunk)
        file_id = (hashobj.digest(), os.path.getsize(full_path))
        duplicate = hashes.get(file_id, None)
        if duplicate:
            print ("Duplicate found: %s and %s" % (full_path, duplicate))
            os.remove(full_path)
        else:
            hashes[file_id] = full_path

In [151]:
for country in CL:
    path = os.path.join('./datasets/', "telegram_Dataset", country)
    #remove_duplicates(path)
    if os.path.exists(path):
        check_for_duplicates(path)

Duplicate found: ./datasets/telegram_Dataset\China\@arabicafrica15134_0_telegram.txt and ./datasets/telegram_Dataset\China\@ShaterGaddafi3455_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\CAR\@marocrus4426_0_telegram.txt and ./datasets/telegram_Dataset\CAR\@bengazi_exit2070_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\Iran\@tangermanar2898_0_telegram.txt and ./datasets/telegram_Dataset\Iran\@marocrus4847_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\Morocco\@marocrus4534_1_telegram.txt and ./datasets/telegram_Dataset\Morocco\@marocrus4534_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\Morocco\@marocrus5065_1_telegram.txt and ./datasets/telegram_Dataset\Morocco\@marocrus5065_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\Morocco\@marocrus5272_1_telegram.txt and ./datasets/telegram_Dataset\Morocco\@marocrus5272_0_telegram.txt
Duplicate found: ./datasets/telegram_Dataset\Morocco\@tangermanar3309_1_telegram.txt and ./data